# 03 - Interactive Viz

## Deadline

Wednesday November 8th, 2017 at 11:59PM


In [1]:
import os
import pandas as pd
import json
import folium
import re
import numpy as np
import branca

pd.options.mode.chained_assignment = None  # default='warn'

# Assignment
## Question 1
 Go to the [eurostat](http://ec.europa.eu/eurostat/data/database) website and try to find a dataset that includes the european unemployment rates at a recent date.

   Use this data to build a [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) which shows the unemployment rate in Europe at a country level. Think about [the colors you use](https://carto.com/academy/courses/intermediate-design/choose-colors-1/), how you decided to [split the intervals into data classes](http://gisgeography.com/choropleth-maps-data-classification/) or which interactions you could add in order to make the visualization intuitive and expressive. Compare Switzerland's unemployment rate to that of the rest of Europe.



In [2]:
def series_str_to_float(s):
    """
    Simple utility function to convert strings to floats
    """
    for idx in s.index:
        try:
            non_decimal = re.compile(r'[^\d.]+')
            s[idx]=non_decimal.sub('',s[idx])
        except:
            pass
        try:
            s[idx]=float(s[idx])
        except:
            s[idx]=np.nan
    return s

### Data extraction
First we extract the unemployement rates for 2016, and only consider the data for the full age span (15 to 74), both sexes (T), and the total citizen status regarding the country (TOTAL).
The choices of these broad parameters allows us to take into account every unemployed person, but the rate does not take into account if the person are employable (min/max age to work) or looking for a job. As these parameters depends on each country rules we will carry on with our general approach.


In [3]:
rate_filepath='./dataEurope/lfsa_urgan.tsv'

def import_file(filepath):
    df2=pd.read_table(filepath)
    s=df2[df2.columns[0]]
    df1=s.str.split(pat=',', expand=True)
    df1.columns=('Unit', 'Sex', 'Age', 'Citizen', 'Country')
    df2.drop(df2.columns[0],axis=1,inplace=True)
    full_df=pd.concat([df1,df2], axis=1)
    #Cleaning the series for the most recent entry (2016)
    s_lastyear=series_str_to_float(full_df[full_df.columns[5]])
    return pd.concat([full_df[['Unit', 'Sex', 'Age', 'Citizen', 'Country']],s_lastyear],axis=1)

df_rate=import_file(rate_filepath)
df_rate.rename(columns={df_rate.columns[5]:'Rate'}, inplace=True)

cropped1=df_rate[df_rate.Age=='Y15-74']
cropped2=cropped1[cropped1.Sex=='T']
cropped3=cropped2[cropped2.Citizen=='TOTAL']

Now we can print the unemployement map in a html file [europeen_unemployement.html](https://rawgit.com/Polymere/ADA_Group/master/Homework/03 - Interactive Viz/htmlGraphOutputs/europeen_unemployement.html)

Colors go from light red to red. Countries where the uneployement rate are missing for 2017 are colored in black. We can clearely see that Swizerland is part of the countries that have the lowest unemployement rate.

In [4]:
HTML_PATH = "htmlGraphOutputs/"

In [5]:
path_EU='./topojson/europe.topojson.json'
eu_json_data=json.load(open(path_EU))
eu_map=folium.Map(location=[46, 7],zoom_start=3)

# Main Choropleth map
eu_map.choropleth(
    geo_data=eu_json_data,
    topojson='objects.europe', 
    data=cropped3, 
    columns=['Country', 'Rate'],
    key_on='feature.id', 
    fill_color='OrRd', 
    fill_opacity=0.8, 
    line_opacity=0.6,
    legend_name='Unemployement rate(%)')

# Aditive layer that blackens countries where the data is missing
colorscale = branca.colormap.linear.OrRd.scale(0, np.max(cropped3['Rate']))
country_index = cropped3.copy()
country_index.index = cropped3['Country']

def style_function(feature):
    id_ = feature['id']
    try:
        color = colorscale(country_index.loc[id_]['Rate'])
        fill_opacity = 0
    except KeyError:
        fill_opacity = 0.5
        color = '#black'
        
    return {
        'fillOpacity': fill_opacity,
        'weight': 0,
        'fillColor': color
    }

folium.TopoJson(
    open(path_EU),
    'objects.europe',
    style_function=style_function
).add_to(eu_map)

eu_map.save(HTML_PATH + "europeen_unemployement.html")

## Question 2
Go to the [amstat](https://www.amstat.ch) website to find a dataset that includes the unemployment rates in Switzerland at a recent date.

   > *HINT* Go to the `details` tab to find the raw data you need. If you do not speak French, German or Italian, think of using free translation services to navigate your way through. 

   Use this data to build another Choropleth map, this time showing the unemployment rate at the level of swiss cantons. Again, try to make the map as expressive as possible, and comment on the trends you observe.

   The Swiss Confederation defines the rates you have just plotted as the number of people looking for a job divided by the size of the active population (scaled by 100). This is surely a valid choice, but as we discussed one could argue for a different categorization.

   Copy the map you have just created, but this time don't count in your statistics people who already have a job and are looking for a new one. How do your observations change ? You can repeat this with different choices of categories to see how selecting different metrics can lead to different interpretations of the same data.
   
   ### Importing the data
   We import the data, only considering the last columns where the yearly total is.

In [6]:
path_CH_data='./dataCH/TauxDeChomageCH.csv'
df=pd.read_csv(path_CH_data,encoding='utf_16le')
df.drop('Grande région',axis=1,inplace=True)
df.drop('Mois', axis=1, inplace=True)
df.rename(columns={df.columns[1]:'Grande région'}, inplace=True)
df.rename(columns={'Total': 'Taux de chômage', 'Total.1':'Chômeurs inscrits' , 
                   'Total.2':'Demandeurs d\'emploi', 'Total.3':'Demandeurs d\'emploi non chômeurs'}, inplace=True)

cr=df[df.columns[[0,1,2,43,44,45,46]]]

cr.drop(cr.index[0], inplace=True)


### Unemployement metrics
The current metric defining the unemployement rate is the number of people looking for a job over the total active population.
However, we see that a non negligible part of the job seekers are not unemployed, meaning that they have a job, while they are looking for a better one.
We will define a new metric which doesn't take into account the employed jobseekers.
The formula to get this new rate would be 
new_rate=unemployed*old_rate/jobseekers 
 

First we start by matching the id's from the topojson file to the one of the data, and then we print the map.

The map with the original rates can be seen here: [swiss unemployement rate](https://rawgit.com/Polymere/ADA_Group/master/Homework/03 - Interactive Viz/htmlGraphOutputs/swiss_unemployement.html)

The map with the new calulated rates can be seen here: [swiss unemployement rate without employed jobseekers](https://rawgit.com/Polymere/ADA_Group/master/Homework/03 - Interactive Viz/htmlGraphOutputs/swissnew_unemployement.html)

As before, the colors go from light red to red, red being the canton with the most unemployement. Comparing the new maps, we observe all cantons loose at least 1% of unemployement, and cantons like Fribourg and Valais have there unemployement rate divided by two.

In [7]:
cr['Taux de chômage']=series_str_to_float(cr['Taux de chômage'])
cr['Chômeurs inscrits']=series_str_to_float(cr['Chômeurs inscrits'])
cr['Demandeurs d\'emploi']=series_str_to_float(cr['Demandeurs d\'emploi'])
cr['Demandeurs d\'emploi non chômeurs']=series_str_to_float(cr['Demandeurs d\'emploi non chômeurs'])

To match the ids, we sort the ids from the data and from the topojson file. This way most of the canton are already matched. For the ones that d not match we do the replacement by hand.

In [8]:
path_CH='./topojson/ch-cantons.topojson.json'

ch_json_data=json.load(open(path_CH))
lst_json_names=[ids['properties']['name'] for ids in ch_json_data['objects']['cantons']['geometries']]
lst_json_names.sort()

cr.sort_values('Canton', inplace=True)
lst_df_names=cr.Canton.values.tolist()
lst_df_names.remove(lst_df_names[26])

df_name=pd.DataFrame(lst_df_names, lst_json_names)

In [9]:
lst_json_names.append('Total')
cr['TradCanton']=lst_json_names
cr['Canton']=cr['TradCanton']

def replace(idx1, idx2):
    cr['Canton'][idx1]=cr['TradCanton'][idx2]

replace(9,10)
replace(10,5)
replace(5,9)
replace(1,3)
replace(4,1)
replace(3,4)
replace(26,13)
replace(13,26)

In [10]:
ch_map=folium.Map(location=[46, 7],zoom_start=6)
ch_map.choropleth(geo_data=ch_json_data,topojson='objects.cantons', data=cr, columns=['Canton','Taux de chômage'],
                 key_on='feature.properties.name', fill_color='OrRd', fill_opacity=0.8, line_opacity=0.6,
             legend_name='Unemployement rate(%)')
ch_map.save(HTML_PATH + "swiss_unemployement.html")

Now we calculate the new rates using the formula discussed above

In [11]:
cr['New_Rate']=cr['Chômeurs inscrits']*cr['Taux de chômage']/cr['Demandeurs d\'emploi']

In [12]:
new_ch_map=folium.Map(location=[46, 7],zoom_start=6)
new_ch_map.choropleth(geo_data=ch_json_data,topojson='objects.cantons', data=cr, columns=['Canton','New_Rate'],
                 key_on='feature.properties.name', fill_color='OrRd', fill_opacity=0.8, line_opacity=0.6,
             legend_name='Unemployement rate(%)')
new_ch_map.save(HTML_PATH + "swissnew_unemployement.html")

## Question 3
 Use the [amstat](https://www.amstat.ch) website again to find a dataset that includes the unemployment rates in Switzerland at recent date, this time making a distinction between *Swiss* and *foreign* workers.

   The Economic Secretary (SECO) releases [a monthly report](https://www.seco.admin.ch/seco/fr/home/Arbeit/Arbeitslosenversicherung/arbeitslosenzahlen.html) on the state of the employment market. In the latest report (September 2017), it is noted that there is a discrepancy between the unemployment rates for *foreign* (`5.1%`) and *Swiss* (`2.2%`) workers. 

   Show the difference in unemployment rates between the two categories in each canton on a Choropleth map (*hint* The easy way is to show two separate maps, but can you think of something better ?). Where are the differences most visible ? Why do you think that is ?

   Now let's refine the analysis by adding the differences between age groups. As you may have guessed it is nearly impossible to plot so many variables on a map. Make a bar plot, which is a better suited visualization tool for this type of multivariate data.


### Loading the data
As in the previous question, we load the data and replace the names to match the names of the json file.


In [14]:
path_CH_data='./dataCH/Arbeitslosenquoten_Herkunft.csv'
df=pd.read_csv(path_CH_data,encoding='utf_16le')

df.rename(columns={'Kanton': 'Canton', 'Nationalität': 'Nationalité',
                   'Gesamt': 'Taux de chômage', 'Gesamt.1':'Chômeurs inscrits' , 
                   'Gesamt.2':'Demandeurs d\'emploi', 'Gesamt.3':'Demandeurs d\'emploi non chômeurs'}, inplace=True)

cr = df[df.columns[[1, 2, 44, 45, 46, 47]]]
cr.drop(cr.index[0], inplace=True)
cr.sort_values(by=["Canton", "Nationalité"], inplace=True)
cr = cr[:-1] # Drop NaN value

cr.replace(to_replace="Bern", value="Bern/Berne", inplace=True)
cr.replace(to_replace="Freiburg", value="Fribourg", inplace=True)
cr.replace(to_replace="Genf", value="Genève", inplace=True)
cr.replace(to_replace="Graubünden", value="Graubünden/Grigioni", inplace=True)
cr.replace(to_replace="Neuenburg", value="Neuchâtel", inplace=True)
cr.replace(to_replace="Tessin", value="Ticino", inplace=True)
cr.replace(to_replace="Waadt", value="Vaud", inplace=True)
cr.replace(to_replace="Wallis", value="Valais/Wallis", inplace=True)

cr['Taux de chômage']=series_str_to_float(cr['Taux de chômage'])
cr['Chômeurs inscrits']=series_str_to_float(cr['Chômeurs inscrits'])
cr['Demandeurs d\'emploi']=series_str_to_float(cr['Demandeurs d\'emploi'])
cr['Demandeurs d\'emploi non chômeurs']=series_str_to_float(cr['Demandeurs d\'emploi non chômeurs'])


### Plotting the differences
We then select differentiate the rates for strangers and swiss and plot them in two different maps

In [ ]:
path_CH='./topojson/ch-cantons.topojson.json'

ch_json_data=json.load(open(path_CH, encoding="utf-8"))
lst_json_names=[ids['properties']['name'] for ids in ch_json_data['objects']['cantons']['geometries']]
lst_json_names.sort()

cr.sort_values('Canton', inplace=True)
lst_df_names=cr.Canton.values.tolist()
lst_df_names.remove(lst_df_names[26])

cr_swiss = cr[1::2]
cr_foreigner = cr[::2]

df_name_swiss=pd.DataFrame(lst_df_names[::2], lst_json_names)

ch_map_swiss=folium.Map(location=[46, 7],zoom_start=6)
ch_map_swiss.choropleth(geo_data=ch_json_data, topojson='objects.cantons', data=cr_swiss, columns=['Canton','Taux de chômage'],
                 key_on='feature.properties.name', fill_color='OrRd', fill_opacity=0.8, line_opacity=0.6,
             legend_name='Unemployement rate(%)')
ch_map_swiss.save(HTML_PATH + "swiss_unemployement_by_nationality_swiss.html")



df_name_etranger=pd.DataFrame(lst_df_names[::2], lst_json_names)


ch_map_etranger=folium.Map(location=[46, 7],zoom_start=6)
ch_map_etranger.choropleth(geo_data=ch_json_data, topojson='objects.cantons', data=cr_foreigner, columns=['Canton','Taux de chômage'],
                 key_on='feature.properties.name', fill_color='OrRd', fill_opacity=0.8, line_opacity=0.6,
             legend_name='Unemployement rate(%)')
ch_map_etranger.save(HTML_PATH + "swiss_unemployement_by_nationality_foreigner.html")

## Question 4 (Bonus)

 *BONUS*: using the map you have just built, and the geographical information contained in it, could you give a *rough estimate* of the difference in unemployment rates between the areas divided by the [RÃ¶stigraben](https://en.wikipedia.org/wiki/R%C3%B6stigraben)?